## Segments

* NAMES - explains the formats
* CMPLDR.DAT - company leader
* CMPOPN.DAT - campaign oponent
* COMPANY.DAT - ????
* DTCHLDR.DAT - detachment leader
* SUPPORT.DAT - detachment makeups
* UNIT.DAT - individual unit
* WEAPON.DAT - weapon stats

regiment has 4 companies
company has leader and 4 detachments
detachment has leader name 1-5 units
unit has 1-4 weapons?

Action Sets? `./SET/*.BIN` and `SET.MUK`

* REGION 1-36

## action sets

describe which set of sprites refer to a certain action, game files has a couple of action set BIN files, the rest are merged into one big SET.MUK file

## buildings

the building files are merged into one big BDG.MUK file



## regions

`mapnum.pcx` with region map and order:

{
    region name,
    resources,
    region info text filename,
    region map name file,
    x,
    y,
    icon coordinates,
    [0-7] neighbouring regions (-1 for empty),
    [0-23] x, y, facing, form
)

where is the pcx pallet index value kept?

## Formats

Company leader {
    name, 
    experience,
    detachement1,
    detachement2,
    detachement3,
    detachement4
}

detachment 1-4 has nationality and unit type.

detachment leader {
    nationality,
    name
}

detachment
{
	ID.
	"Title Line One".
   "Title Line Two".
	NUM_UNIT1 UNIT_ID1.
	...
	NUM_UNIT5 UNIT_ID5.
	Break. Morale. VictoryPoints.
	Cost.
	LIconOffset. RIconOffset.NationalityOffset.
	Flags.
   AllowableNationalityFlag.
   AllowTypeFlag.
   StartExperience.
   Turns(time to build). !!!!!for now is   Cost / 200
}

In [1]:
import os
from collections import defaultdict
import logging
from IPython.display import HTML, display
import math
from itertools import islice, product
from enum import Enum

In [2]:
nationalities = {
    # in the hpp these had an extra WORD with zeros?
    # used in the company ledaers but not the detach leaders
    0x0001: "FLAG_NATION_TALLARN",
    0x0002: "FLAG_NATION_MORDIAN",
    0x0004: "FLAG_NATION_CATACHAN",
    0x0007: "FLAG_NATION_T_M_C",
    0x0008: "FLAG_NATION_ATTILAN",
    0x0010: "FLAG_NATION_SPACEMARINE",
    0x0020: "FLAG_NATION_FORGEWORLD",
    0x0040: "FLAG_NATION_PLANETARY",
    0x00FF: "FLAG_NATION_ANYIMPERIAL",
    0x0100: "FLAG_NATION_GOFF",
    0x0200: "FLAG_NATION_BAD_MOON",
    0x0400: "FLAG_NATION_EVIL_SUNZ",
    0x0800: "FLAG_NATION_MEKBOY",
    0x0C00: "FLAG_NATION_ES_OR_MEK",
    0x0900: "FLAG_NATION_GOFF_OR_MEK",
    0x0a00: "FLAG_NATION_BAD_OR_MEK",
    0xff00: "FLAG_NATION_ANYORK",
    0xffff: "FLAG_NATION_ANYNATION",
}

unit_types = {
    0xffffffff: "FLAG_TYPE_ANYTYPE",
    0x00010000: "FLAG_TYPE_INFANTRY_WALKER",
    0x00020000: "FLAG_TYPE_CAVALRY",
    0x00100000: "FLAG_TYPE_KNIGHTS",
    0x00080000: "FLAG_TYPE_TANKS",
    0x00040000: "FLAG_TYPE_SUPER_HEAVY",
    0x000c0000: "FLAG_TYPE_ANY_ARMOR",
    0x00200000: "FLAG_TYPE_TITAN",
    0x00400000: "FLAG_TYPE_FLYER",
    0x00800000: "FLAG_TYPE_BIKES",
    0x00880000: "FLAG_TYPE_TANK_OR_BIKE",
    0x008c0000: "FLAG_TYPE_ANY_TANK_OR_BIKE"
}

In [3]:
def chunk(it, size):
    it = iter(it)
    return list(iter(lambda: tuple(islice(it, size)), ()))


def format_word(v, w):
    text = '{:02X} {:02X}'.format(v, w)
    return f'{text}'

    
def hex_view(data):
    text = []
    text.append(
        14 * '&nbsp;' +
        '&nbsp;'.join(['{:02X}'.format(y) for y in range(8)]) +
        '&nbsp;&nbsp;' +
        '&nbsp;'.join(['{:02X}'.format(y) for y in range(8, 16)])
    
    )
    text.append('')
    for i, x in enumerate(chunk(data, 16)):
        text.append('{:08X}:&nbsp;&nbsp;&nbsp;&nbsp;'.format(i * 16) +
              '&nbsp;'.join([format_word(y, z) for y, z in zip(x[:8:2], x[1:8:2])]) +
              '&nbsp;&nbsp;' +
              '&nbsp;'.join([format_word(y, z) for y, z in zip(x[8:16:2], x[9:16:2])])
             )
    return '<span style="font-family: Courier New, Courier, Lucida Sans Typewriter, Lucida Typewriter, monospace;">{}</span>'.format("<br>".join(text))


def display_hex_view(data):
    display(HTML(hex_view(data)))


display_hex_view(bytes(range(132)))

In [4]:
# Look at the top of the files
FOLDER = '/home/alex/.wine/drive_c/GOG Games/WH40K Final Liberation/DAT'
OUTPUT = './sprites/'

for filepath in sorted(os.listdir(FOLDER)):
    filesize = os.path.getsize(FOLDER + '/' + filepath)
    filename = filepath.split('.')[0]
    print(filename, filesize)

CMPLDR 4384
CMPOPN 404479
COMPANY 1387
DTCHLDR 2300
SUPPORT 5102
UNIT 12073
WEAPON 3124


In [5]:
with open(f'{FOLDER}/DTCHLDR.DAT', 'rb') as file:
    leader_count = int.from_bytes(file.read(2), 'little')
    blank = file.read(4)
    deatchment_leaders = []
    for i in range(leader_count):
        nation = int.from_bytes(file.read(2), 'little')
        deatchment_leaders.append((
            nationalities.get(nation, hex(nation)), 
            bytes([x for x in file.read(26) if x != 0x00]).decode('utf-8')
        ))
for x in deatchment_leaders:
    print(x)

('FLAG_NATION_TALLARN', 'Arif')
('FLAG_NATION_TALLARN', 'Abdul Salam')
('FLAG_NATION_TALLARN', 'Azhara')
('FLAG_NATION_TALLARN', 'Rahimat')
('FLAG_NATION_TALLARN', 'Asim')
('FLAG_NATION_TALLARN', 'Tahir Majnun')
('FLAG_NATION_TALLARN', 'Jabir')
('FLAG_NATION_MORDIAN', 'Fredrick')
('FLAG_NATION_MORDIAN', 'Ulrich')
('FLAG_NATION_MORDIAN', 'Dietrich')
('FLAG_NATION_MORDIAN', 'Hans Bernd')
('FLAG_NATION_MORDIAN', 'Claus Schenk')
('FLAG_NATION_MORDIAN', 'Kluge')
('FLAG_NATION_MORDIAN', 'Konstantin')
('FLAG_NATION_CATACHAN', 'Pelayo')
('FLAG_NATION_CATACHAN', 'Rodrigo')
('FLAG_NATION_CATACHAN', 'Diaz')
('FLAG_NATION_CATACHAN', 'Jimena')
('FLAG_NATION_CATACHAN', 'Comuneros')
('FLAG_NATION_CATACHAN', 'Acuna')
('FLAG_NATION_CATACHAN', 'Giron')
('FLAG_NATION_SPACEMARINE', 'Theonoe')
('FLAG_NATION_SPACEMARINE', 'Creon')
('FLAG_NATION_SPACEMARINE', 'Andros')
('FLAG_NATION_SPACEMARINE', 'Sybaris')
('FLAG_NATION_SPACEMARINE', 'Dorian')
('FLAG_NATION_SPACEMARINE', 'Iphicrates')
('FLAG_NATION_SPACEMAR

In [21]:
# first and third bytes (and some of the 4th byte) contain flags
# second byte contains titan weapon location flags
# 4th byte contains a nibble for special weapon type
weapon_flags = {
    0x00000010: "FLAK_STYLE",
    0x00000020: "THIRD_MIN_RANGE",
    0x00000040: "ONE_PER_DETACHMENT", 
    0x00000080: "CAN_TARGET_ANY_LOCATION",
    0x00000100: "PLUS_ONE_MELEE_BONUS",
    0x00000200: "PLUS_DIE_MELEE_BONUS",
    0x00000400: "MELEE_DAMAGE_BONUS",
    0x00000800: "MELEE_CRIT_BONUS",
    0x00001000: "OPTIONAL_AMMO",
    0x00002000: "OPTIONAL_AMMO_LISTING",
    0x00004000: "OBSERVATION_HIT_BONUS",
    0x00008000: "MORALE_BONUS",
    0x40000000: "SPECIAL_WEAPON", 
    0x80000000: "TEMPLATE_WEAPON", 
    0x20000000: "BARRAGE_WEAPON", 
    0x08000000: "FW_IGNORES_COVER_MOD",
    0x04000000: "FW_TURN_TO_RELOAD"
}
hit_location = {
    # Weapon Location specifiers needed for all weapons which can be mounted
    0x00010000: "TITAN_GENERAL_AREA",
    0x00020000: "TITAN_RIGHT_ARM_AREA",
    0x00040000: "TITAN_LEFT_ARM_AREA",
    #"TITAN_BOTH_ARM_AREA"				0x00060000
    0x00080000: "TITAN_RIGHT_CARAP_AREA",
    0x00100000: "TITAN_LEFT_CARAP_AREA",
    #"TITAN_BOTH_CARAP_AREA"			0x00180000
    0x00200000: "TITAN_HEAD_AREA",
    0x00400000: "TITAN_BELLY_AREA",
    # "TITAN_AREA_ALL"						0x007F0000

    # this is a special area because the harpoon missle can be attached to
    # both the launcher and a titan carapace area
    0x00080000: "LAUNCHER_AREA",
    0x00010000: "GENERAL_AREA"
}

# first nibble contains a special weapon type if the flag is set
special_weapons = {
    0: "TEMPLATE_TYPE_PULSA",
    1: "TEMPLATE_TYPE_BARRAGE_MISSLE",
    2: "TEMPLATE_TYPE_WARP",
    3: "TEMPLATE_TYPE_FLAMER",
    4: "TEMPLATE_TYPE_VORTEX",
    5: "TEMPLATE_TYPE_SNAPPER",
    6: "TEMPLATE_TYPE_KUSTOM_ROKKITS",
    11: "SPECIAL_TYPE_LIFTA",
    12: "SPECIAL_TYPE_WIERDBOY",
    13: "SPECIAL_TYPE_KUST_KANNON"
}

In [26]:
with open(f'{FOLDER}/WEAPON.DAT', 'rb') as file:
    header = file.read(12)
    count = int.from_bytes(file.read(4), 'little')
    print(count)
    for i in range(count):
        flags = int.from_bytes(file.read(4), 'little')
#         assert flags & (
#             0xffffffff - sum(weapon_flags.keys())
#         ) == 0, f"Unknown flag, {i} {flags}"
        flag_text = []
        for f, v in weapon_flags.items():
            if f & flags:
                flag_text.append(v)
        if 'SPECIAL_WEAPON' in flag_text or' TEMPLATE_WEAPON' in flag_text:
            special_type = special_weapons[flags & 0xf]
        else:
            special_type = ''
        titan_mount_areas = []
        for f, v in hit_location.items():
            if f & flags:
                titan_mount_areas.append(v)
        weapon_id = int.from_bytes(file.read(2), 'little')
        sound_id = int.from_bytes(file.read(2), 'little')
        visual_id = int.from_bytes(file.read(2), 'little')
        trails_id = int.from_bytes(file.read(2), 'little')
        weapon_range = int.from_bytes(file.read(1), 'little', signed=True)
        penetration = int.from_bytes(file.read(1), 'little', signed=True)
        rate_of_fire = int.from_bytes(file.read(1), 'little')
        unknown2 = int.from_bytes(file.read(1), 'little')
        assert unknown2 == 0, f"Unknown value is not zero {unknown2}"
        name = bytes([x for x in file.read(26) if x != 0x00]).decode('utf-8')
        print(i, weapon_id, name, sound_id, visual_id, 
              trails_id, 
              "Stats: ", weapon_range, penetration, 
              rate_of_fire
             )
        print(
              flag_text,
              special_type,
             titan_mount_areas)
        print()
    assert file.tell() == os.fstat(file.fileno()).st_size, "Did not read to EOF"

74
0 1001 Autocannon 2001 881 0 Stats:  20 60 2
[]  []

1 1002 Axes 0 992 0 Stats:  0 25 1
[]  []

2 1003 Barrage Missile 2042 1105 880 Stats:  80 75 1
['TEMPLATE_WEAPON', 'BARRAGE_WEAPON']  ['LAUNCHER_AREA']

3 1004 Battlecannon 2005 882 0 Stats:  20 75 1
[]  []

4 1005 Battlecannon 2005 882 0 Stats:  28 75 1
[]  []

5 1006 Bolter(s), Tank 2036 886 0 Stats:  8 40 2
[]  []

6 1007 Bolters, Pistol 2002 886 0 Stats:  10 40 2
[]  []

7 1008 Bolters, Bike 2036 886 0 Stats:  10 55 2
[]  []

8 1009 Bolters 2002 886 0 Stats:  20 40 4
[]  []

9 1010 Bolters, Heavy 2036 886 0 Stats:  20 50 2
[]  []

10 1011 Bubble Chukka 2047 3107 0 Stats:  20 -20 1
[]  ['TITAN_RIGHT_ARM_AREA']

11 1012 Buzz Saw Arm 2001 882 0 Stats:  20 75 1
['PLUS_DIE_MELEE_BONUS']  ['TITAN_RIGHT_ARM_AREA']

12 1014 Chain Fist 2007 992 0 Stats:  0 120 1
['PLUS_DIE_MELEE_BONUS']  ['TITAN_RIGHT_ARM_AREA']

13 1015 Cluster Buster 2013 882 0 Stats:  28 70 2
[]  ['TITAN_RIGHT_ARM_AREA', 'TITAN_LEFT_ARM_AREA']

14 1016 D'Day Cannon

In [24]:
unit_flags = {
    0x00000100: "FLAG_UNIT_HAS_CRITS",
    0x00000200: "FLAG_UNIT_CHOOSE",
    0x00000400: "FLAG_NO_ADVANCE",
    0x00000800: "FLAG_HAS_FOOTPRINT",
    0x00001000: "FLAG_SPECIAL_SPEED",
    0x00002000: "FLAG_UNIT_TRANSPORT",
    0x00004000: "FLAG_OFFBOARD",
    0x00008000: "FLAG_MELEE_BONUS",
    0x00010000: "FLAG_MELEE_OVERRUN",
    0x00020000: "FLAG_STUBBORN",
    0x00040000: "FLAG_SEPARATE_MELEE",
    0x00080000: "FLAG_LEAVES_NOBLOCKAGE",
    0x10000000: "FLAG_UNIT_MISSLE",
    0x20000000: "FLAG_UNIT_MORALE_BONUS",
    0x40000000: "FLAG_UNIT_JET_PACKS",
}


class Shields(Enum):
    VOID_SHIELD = 1
    POWER_FIELD = 2
    GARGANT_POWER_FIELD = 3
    MEKBOY_KUSTOM_FIELD = 4

class Criticals(Enum):
    CRIT_DAMAGE = -1
    CRIT_NOTHING = 0
    CRIT_MOVEMENT = 1
    CRIT_WEAPONS = 2
    CRIT_SYSTEMS = 3
    CRIT_TRANSPORTS = 4

class Severity(Enum):
    CRIT_DAMAGED = 0
    CRIT_DESTROYED = 1

In [8]:
def read_unit(file, unit_lookup):
    uid = int.from_bytes(file.read(2), 'little')
    data1 = int.from_bytes(file.read(2), 'little')
    uid2 = int.from_bytes(file.read(2), 'little')  # shadow ids for flyers?
    data2 =int.from_bytes(file.read(2), 'little')
    data4 = file.read(1)
    iid = int.from_bytes(file.read(4), 'little')  # icon id
    name_length = int.from_bytes(file.read(1), 'little')
    name = bytes(file.read(name_length)).decode('utf-8')
    move = int.from_bytes(file.read(1), 'little')
    save = int.from_bytes(file.read(1), 'little')
    caf = int.from_bytes(file.read(1), 'little')
    armours = []
    for i in range(4):
        unknown = int.from_bytes(file.read(1), 'little')
        #assert unknown == 0, "Initial armour value should be zero"
        d6_value = int.from_bytes(file.read(1), 'little')
        armour_front = int.from_bytes(file.read(1), 'little')
        armour_side = int.from_bytes(file.read(1), 'little')
        armour_rear = int.from_bytes(file.read(1), 'little')
        hit_offset_x = int.from_bytes(file.read(2), 'little', signed=True)
        hit_offset_y = int.from_bytes(file.read(2), 'little', signed=True)
        hit_offset_z = int.from_bytes(file.read(2), 'little', signed=True)
        armours.append(
            (unknown, d6_value, armour_front, armour_side, armour_rear, 
             (hit_offset_x, hit_offset_y, hit_offset_z)))
    chassis_type = int.from_bytes(file.read(1), 'little')
    weapon_slots = int.from_bytes(file.read(1), 'little')
    shield_type = int.from_bytes(file.read(1), 'little')
    shields = int.from_bytes(file.read(1), 'little')
    hit_points = int.from_bytes(file.read(1), 'little')
    unit_cost = int.from_bytes(file.read(2), 'little')
    
    # critical hits (min 56 bytes)
    unknown2 = int.from_bytes(file.read(56), 'little')
#     crit_type = int.from_bytes(file.read(1), 'little')
#     dx = int.from_bytes(file.read(1), 'little', signed=True)
#     dy = int.from_bytes(file.read(1), 'little', signed=True)
#     dz = int.from_bytes(file.read(1), 'little', signed=True)
#     crits = []
#     for i in range(11):
#         crits.append(file.read(1))
    
    # weaps
    weapons = []
    for i in range(weapon_slots):
        weapons.append(
            {'weapon_id': int.from_bytes(file.read(2), 'little')}
        )
    for i in range(weapon_slots):
        weapons[i]['weapon_count'] = int.from_bytes(file.read(1), 'little')
    for i in range(weapon_slots):
        weapons[i]['base_hit_chance'] = int.from_bytes(file.read(1), 'little')
    for i in range(weapon_slots):
        dx = int.from_bytes(file.read(1), 'little', signed=True)
        dy = int.from_bytes(file.read(1), 'little', signed=True)
        dz = int.from_bytes(file.read(1), 'little', signed=True)
        weapons[i]['offset'] = (dx, dy, dz)
        term = int.from_bytes(file.read(1), 'little', signed=True)
        #assert term == 0, "final byte in weapon slot should be zero"
    unknown3 = int.from_bytes(file.read(6), 'little')
    unit_lookup[uid] = name
    unit = {
        "unit_id": uid, 
        "unit_id2": uid2, 
        "unknown": (data1, data2, data4), 
        "icon_id": iid, 
        "name": name, 
        "move": move, 
        "save": save, 
        "caf": caf,
        "armour": armours,
        "chassis_type": chassis_type,
        "weapon_slots": weapon_slots,
        "weapons": weapons,
        "shield_type": shield_type,
        "shields": shields,
        "hit_points": hit_points,
        "unit_cost": unit_cost,
        "unknown2": unknown2,
        "unknown3": unknown3,
        "flags": None,
        "troop_transport": None,
    }
    print(unit)

In [10]:
unit_lookup = {}
with open(f'{FOLDER}/UNIT.DAT', 'rb') as file:
    header = file.read(13)
    count = int.from_bytes(file.read(4), 'little')
    print(count)
    for i in range(count):
        read_unit(file, unit_lookup)
        print()
    assert file.tell() == os.fstat(file.fileno()).st_size, "Did not read to EOF"

78
{'unit_id': 1504, 'unit_id2': 2928, 'unknown': (0, 4, b'\x00'), 'icon_id': 8000, 'name': 'Warlord', 'move': 6, 'save': 0, 'caf': 14, 'armour': [(0, 2, 90, 75, 60, (-30, -10, 20)), (0, 3, 120, 90, 60, (0, -30, 80)), (0, 4, 100, 80, 60, (-35, -30, 90)), (0, 6, 85, 75, 65, (-20, -20, 100))], 'chassis_type': 15, 'weapon_slots': 4, 'weapons': [{'weapon_id': 65535, 'weapon_count': 1, 'base_hit_chance': 50, 'offset': (-40, -45, 90)}, {'weapon_id': 65535, 'weapon_count': 1, 'base_hit_chance': 50, 'offset': (0, 0, 0)}, {'weapon_id': 65535, 'weapon_count': 1, 'base_hit_chance': 50, 'offset': (-33, -35, 125)}, {'weapon_id': 65535, 'weapon_count': 1, 'base_hit_chance': 50, 'offset': (28, -35, 125)}], 'shield_type': 1, 'shields': 6, 'hit_points': 12, 'unit_cost': 600, 'unknown2': 11390170936755232855409966481497056395322628287910367335055156988200663514753157426419391150500879736094800271694618531422819848007662, 'unknown3': 196610, 'flags': None, 'troop_transport': None}

{'unit_id': 1502, 'uni

{'unit_id': 1112, 'unit_id2': 1104, 'unknown': (0, 0, b'\x00'), 'icon_id': 16000, 'name': 'Manticore', 'move': 6, 'save': 5, 'caf': 0, 'armour': [(0, 2, 45, 45, 45, (-14, -10, 5)), (0, 5, 70, 52, 35, (-12, -12, 17)), (0, 6, 50, 50, 50, (0, 0, 22)), (0, 0, 0, 0, 0, (0, 0, 0))], 'chassis_type': 4, 'weapon_slots': 2, 'weapons': [{'weapon_id': 1056, 'weapon_count': 2, 'base_hit_chance': 40, 'offset': (0, -30, 20)}, {'weapon_id': 1056, 'weapon_count': 2, 'base_hit_chance': 40, 'offset': (0, -30, 20)}], 'shield_type': 0, 'shields': 0, 'hit_points': 1, 'unit_cost': 30, 'unknown2': 0, 'unknown3': 0, 'flags': None, 'troop_transport': None}

{'unit_id': 1114, 'unit_id2': 80, 'unknown': (0, 0, b'\x00'), 'icon_id': 16800, 'name': 'Predator', 'move': 10, 'save': 3, 'caf': 0, 'armour': [(0, 1, 50, 50, 50, (-14, -10, 5)), (0, 3, 80, 60, 40, (-12, -12, 17)), (0, 4, 65, 60, 50, (-4, -10, 22)), (0, 6, 85, 62, 40, (0, -5, 26))], 'chassis_type': 4, 'weapon_slots': 3, 'weapons': [{'weapon_id': 1001, 'weapo

{'unit_id': 3116, 'unit_id2': 4176, 'unknown': (0, 0, b'\x00'), 'icon_id': 44000, 'name': 'Lifta Droppa', 'move': 8, 'save': 255, 'caf': 2, 'armour': [(0, 1, 40, 40, 40, (-10, -16, 5)), (0, 4, 70, 53, 35, (-10, -13, 17)), (0, 6, 50, 45, 40, (0, 2, 23)), (0, 0, 0, 0, 0, (0, 0, 0))], 'chassis_type': 8, 'weapon_slots': 1, 'weapons': [{'weapon_id': 1039, 'weapon_count': 1, 'base_hit_chance': 40, 'offset': (0, -20, 25)}], 'shield_type': 0, 'shields': 0, 'hit_points': 1, 'unit_cost': 50, 'unknown2': 0, 'unknown3': 0, 'flags': None, 'troop_transport': None}

{'unit_id': 3118, 'unit_id2': 80, 'unknown': (0, 0, b'\x00'), 'icon_id': 44800, 'name': 'Lungbursta', 'move': 8, 'save': 3, 'caf': 1, 'armour': [(0, 1, 55, 55, 55, (-10, -16, 5)), (0, 5, 90, 65, 45, (-10, -13, 13)), (0, 6, 70, 70, 70, (0, -5, 15)), (0, 0, 0, 0, 0, (0, 0, 0))], 'chassis_type': 10, 'weapon_slots': 1, 'weapons': [{'weapon_id': 1004, 'weapon_count': 1, 'base_hit_chance': 40, 'offset': (0, -18, 12)}], 'shield_type': 0, 'shield

In [11]:
support_flags = {
    0x1: "FLAG_CARD_SPECIAL",
    0x2: "FLAG_CARD_OFFBOARD",
    0x8: "FLAG_SPECIAL_BEHAVIOR",
    0x4000: "FLAG_CARD_ONE_PER_COMPANY",
    0x2000: "FLAG_CARD_ONE_PER_ARMY",
}
ork_flags = {
    0x010000: "FLAG_CARD_GOFF",
    0x020000: "FLAG_CARD_BAD_MOON",
    0x030000: "FLAG_CARD_EVIL_SUNZ",
    0x040000: "FLAG_CARD_MEKBOY",
    0x050000: "FLAG_CARD_KULT_OF_SPEED",
    0x060000: "FLAG_CARD_NOBZ_WARBIKES",
    0x080000: "FLAG_ORK_ACTUAL_VALUES",
    0xff0000: "FLAG_CARD_CLAN_MASK",
}

In [12]:
detachments = {}

# add flags
# add icon lookups
with open(f'{FOLDER}/SUPPORT.DAT', 'rb') as file:
    header = file.read(13)
    count = int.from_bytes(file.read(4), 'little')
    print(count)
    for i in range(count):
        cid = int.from_bytes(file.read(2), 'little')
        flags = int.from_bytes(file.read(4), 'little')
        assert flags & (0xffff - 0x600b) == 0, f"Unknown flag, {cid} {flags}"
        flag_text = []
        for f, v in support_flags.items():
            if f & flags:
                flag_text.append(v)
        licon = hex(int.from_bytes(file.read(4), 'little'))
        ricon = hex(int.from_bytes(file.read(4), 'little'))
        nation = hex(int.from_bytes(file.read(4), 'little'))
        string_length1 = int.from_bytes(file.read(1), 'little')
        string1 = bytes(file.read(string_length1)).decode('utf-8')
        string_length2 = int.from_bytes(file.read(1), 'little')
        string2 = bytes(file.read(string_length2)).decode('utf-8')
        cost = int.from_bytes(file.read(2), 'little')
        breakpoint = int.from_bytes(file.read(1), 'little')
        #define CLAN_MORALE						-1
        #define MORALE_SPECIAL					-2
        #define ALLWAYS_PASS_MORALE			0
        morale = int.from_bytes(file.read(1), 'little', signed=True)
        vp = int.from_bytes(file.read(1), 'little')
        u_count = int.from_bytes(file.read(1), 'little')
        allow_nation = int.from_bytes(file.read(4), 'little') # allow nat flag
        allow_type = int.from_bytes(file.read(4), 'little') # allow type flag
        turns_to_build = int.from_bytes(file.read(2), 'big')
        uids = [int.from_bytes(file.read(2), 'little') for i in range(u_count)]
        unames = [unit_lookup[x] for x in uids]
        detachments[cid] = string1
        print({
            "id": cid, 
            "name1": string1, 
            "name2": string2,
            "cost": cost, 
            "breakpoint": breakpoint, 
            "morale": morale, 
            "victory_points": vp, 
            "units": unames,
            "flags": ",".join(flag_text), 
            "licon": licon, 
            "ricon": ricon, 
            "nation": nation, 
            "allow_nation": nationalities.get(allow_nation >> 16, hex(allow_nation >> 16)), 
            "type": unit_types.get(allow_type, hex(allow_type)), 
            "time_to_build": turns_to_build
        })
        print()
    assert file.tell() == os.fstat(file.fileno()).st_size, "Did not read to EOF"

78
{'id': 11100, 'name1': 'Commissar', 'name2': 'Imperial Guard', 'cost': 0, 'breakpoint': 0, 'morale': 0, 'victory_points': 0, 'units': ['Commissar', 'Rhino'], 'flags': 'FLAG_CARD_SPECIAL,FLAG_CARD_ONE_PER_COMPANY', 'licon': '0x0', 'ricon': '0xc80', 'nation': '0x0', 'allow_nation': 'FLAG_NATION_T_M_C', 'type': 'FLAG_TYPE_INFANTRY_WALKER', 'time_to_build': 0}

{'id': 11123, 'name1': 'Leviathan', 'name2': '', 'cost': 200, 'breakpoint': 0, 'morale': 4, 'victory_points': 2, 'units': ['Leviathan'], 'flags': '', 'licon': '0x0', 'ricon': '0xc80', 'nation': '0x0', 'allow_nation': 'FLAG_NATION_T_M_C', 'type': 'FLAG_TYPE_SUPER_HEAVY', 'time_to_build': 1}

{'id': 11122, 'name1': 'BaneBlade', 'name2': '', 'cost': 200, 'breakpoint': 0, 'morale': 4, 'victory_points': 2, 'units': ['BaneBlade'], 'flags': '', 'licon': '0x0', 'ricon': '0xc80', 'nation': '0x0', 'allow_nation': 'FLAG_NATION_T_M_C', 'type': 'FLAG_TYPE_SUPER_HEAVY', 'time_to_build': 1}

{'id': 11156, 'name1': 'Rhino', 'name2': '', 'cost': 

In [13]:
with open(f'{FOLDER}/CMPLDR.DAT', 'rb') as file:
    leader_count = int.from_bytes(file.read(4), 'little')
    for i in range(leader_count):
        name = bytes([x for x in file.read(24) if x != 0x00]).decode('utf-8')
        exp = int.from_bytes(file.read(1), 'little')
        data = []
        for i in range(4):
            n = int.from_bytes(file.read(4), 'little')
            data.append({
                'nation': nationalities.get(n >> 16, hex(n >> 16))
            })
        for i in range(4):
            u = int.from_bytes(file.read(4), 'little')
            data[i]['unit_type'] = unit_types.get(u, hex(u))
        for i in range(4):
            default = int.from_bytes(file.read(4), 'little')
            data[i]['default_unit'] = detachments[default]
        print(name, exp)
        for x in data:
            print(x)
        print()
    assert file.tell() == os.fstat(file.fileno()).st_size, "Did not read to EOF"

Horgrawl 10
{'nation': 'FLAG_NATION_ANYIMPERIAL', 'unit_type': 'FLAG_TYPE_ANYTYPE', 'default_unit': 'Reaver Titan'}
{'nation': 'FLAG_NATION_ANYIMPERIAL', 'unit_type': 'FLAG_TYPE_ANYTYPE', 'default_unit': 'Leman Russ Squadron'}
{'nation': 'FLAG_NATION_ANYIMPERIAL', 'unit_type': 'FLAG_TYPE_ANYTYPE', 'default_unit': 'Leman Russ Squadron'}
{'nation': 'FLAG_NATION_ANYIMPERIAL', 'unit_type': 'FLAG_TYPE_ANYTYPE', 'default_unit': 'Bombard Battery'}

Jericho 6
{'nation': 'FLAG_NATION_MORDIAN', 'unit_type': 'FLAG_TYPE_INFANTRY_WALKER', 'default_unit': 'Mordian Command Pltn'}
{'nation': 'FLAG_NATION_MORDIAN', 'unit_type': 'FLAG_TYPE_ANY_ARMOR', 'default_unit': 'BaneBlade'}
{'nation': 'FLAG_NATION_MORDIAN', 'unit_type': 'FLAG_TYPE_ANYTYPE', 'default_unit': 'Chimera Squadron'}
{'nation': 'FLAG_NATION_MORDIAN', 'unit_type': 'FLAG_TYPE_ANYTYPE', 'default_unit': 'Bombard Battery'}

Pyrhus 5
{'nation': 'FLAG_NATION_MORDIAN', 'unit_type': 'FLAG_TYPE_INFANTRY_WALKER', 'default_unit': 'Mordian Command Plt